In [1]:
import numpy as np

In [14]:
def dtw(costs):
    """
    move set each has cost 1
    - horizontal
    - vertical
    - diagonal
    """
    rows, cols = costs.shape
    mem = np.zeros((rows, cols), dtype='int')
    for row in range(rows):
        for col in range(cols):
            mem[row, col] = costs[row, col]
    for row in range(rows):
        for col in range(cols):
            if row == 0 and col == 0:
                continue
            elif row == 0:
                mem[row, col] += mem[row, col - 1]
            elif col == 0:
                mem[row, col] += mem[row - 1, col]
            else:
                mem[row, col] += min(mem[row, col - 1],
                                    mem[row - 1, col],
                                    mem[row - 1, col - 1])
    row, col = rows - 1, cols - 1 # destination
    path = [(row, col)]
    # traceback: 0 for horizontal, 1 for vertical, 2 for diagonal
    while True:
        if row == 0 and col == 0:
            break
        if row == 0:
            tb = 0 # horizontal
        elif col == 0:
            tb = 1 # vertical
        else:
            prev = mem[row, col] - costs[row, col]
            if prev == mem[row, col - 1]:
                tb = 0 # horizontal
            elif prev == mem[row - 1, col]:
                tb = 1 # vertical
            else:
                tb = 2 # diagonal
        # after tracing back, update position on board
        if tb == 0:
            col -= 1
        elif tb == 1:
            row -= 1
        else:
            col -= 1
            row -= 1
        path.append((row, col))
    return mem[rows - 1, cols - 1], path[::-1]

In [79]:
def dtw_moveset(costs):
    """
    move set
    - horizontal cost 1
    - diagonal cost 2 => (row + 1, col + 1)
    - two-step diagonal cost 1 => (row + 2, col + 1)
    """
    rows, cols = costs.shape
    mem = np.zeros((rows, cols))
    # initialize all (row>0, col=0) to infinity cost
    mem[0, 0] = costs[0, 0]
    for row in range(1, rows):
        mem[row, 0] = float('inf')
    # DTW algo
    for row in range(rows):
        for col in range(1, cols): # cannot have (row, col=0)
            if row == 0:
                mem[row, col] = costs[row, col] + mem[row, col - 1] # horizontal
            elif row == 1:
                mem[row, col] = \
                min(costs[row, col] + mem[row, col - 1], # horizontal
                    2 * costs[row, col] + mem[row - 1, col - 1]) # diagonal
            else:
                mem[row, col] = \
                min(costs[row, col] + mem[row, col - 1], # horizontal
                    2 * costs[row, col] + mem[row - 1, col - 1], # diagonal
                    costs[row, col] + mem[row - 2, col - 1]) # two-step diagonal
    print(mem)
    # backtrack
    row, col = rows - 1, cols - 1 # at destination
    path = [(row, col)]
    # traceback: 0 for horizontal, 1 for vertical, 2 for diagonal
    while True:
        if row == 0 and col == 0:
            break
        elif col == 0:
            raise # ERROR: how did we get here?
        elif row == 0:
            tb = 0 # horizontal
        elif row == 1: # horizontal or diagonal?
            # cannot have (row, col=0)
            if mem[row, col] - costs[row, col] == mem[row, col - 1] and col != 1:
                tb = 0 # horizontal
            elif mem[row, col] - 2 * costs[row, col] == mem[row - 1, col - 1]:
                tb = 1 # diagonal
            else:
                raise            
        else:
            prev = mem[row, col] - costs[row, col]
            if prev == mem[row, col - 1] and col != 1: # cannot have (row, col=0)
                tb = 0 # horizontal
            elif mem[row, col] - 2 * costs[row, col] == mem[row - 1, col - 1]:
                tb = 1 # diagonal
            elif prev == mem[row - 2, col - 1]:
                tb = 2 # two-step diagonal
            else:
                raise
        # after tracing back, update position on board
        if tb == 0:
            col -= 1
        elif tb == 1:
            row -= 1
            col -= 1
        else:
            row -= 2
            col -= 1
        path.append((row, col))
    return int(mem[rows - 1, cols - 1]), path[::-1]

In [80]:
small = np.array([
    [1, 1, 3],
    [1, 1, 1],
    [2, 1, 2]
])
dtw_moveset(small)

[[ 1.  2.  5.]
 [inf  3.  4.]
 [inf  2.  4.]]


(4, [(0, 0), (2, 1), (2, 2)])

In [81]:
mat = np.array([
    [1, 1, 3, 3, 3, 3],
    [1, 1, 1, 1, 2, 3],
    [2, 1, 2, 2, 1, 3],
    [2, 2, 2, 1, 2, 2],
    [3, 2, 3, 1, 1, 1],
    [3, 2, 3, 2, 2, 2]
])
dtw_moveset(mat)

[[ 1.  2.  5.  8. 11. 14.]
 [inf  3.  4.  5.  7. 10.]
 [inf  2.  4.  6.  7. 10.]
 [inf inf  5.  5.  7.  9.]
 [inf inf  5.  5.  6.  7.]
 [inf inf inf  7.  7.  9.]]


(9, [(0, 0), (1, 1), (1, 2), (3, 3), (5, 4), (5, 5)])